In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import os
path_data = "data/"

In [2]:
raw_data = pd.read_csv(path_data+"AirQualityUCI.csv",engine='python',sep=";",na_values="-200",skipfooter=114)

In [3]:
columns = raw_data.columns[:-2].tolist()

In [4]:
raw_data[columns].dropna()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,30/04/2004,20.00.00,"4,4",1449.0,501.0,"19,5",1282.0,254.0,625.0,133.0,2100.0,1569.0,"19,1","61,1","1,3345"
1227,30/04/2004,21.00.00,"3,1",1363.0,234.0,"15,1",1152.0,189.0,684.0,110.0,1951.0,1495.0,"18,2","65,4","1,3529"
1228,30/04/2004,22.00.00,3,1371.0,212.0,"14,6",1136.0,174.0,689.0,102.0,1927.0,1471.0,"18,1","66,1","1,3579"
1229,30/04/2004,23.00.00,"3,1",1406.0,275.0,"13,7",1107.0,167.0,718.0,108.0,1872.0,1384.0,"17,7","66,9","1,3422"


In [5]:
data_cleaning_1 = raw_data[columns]
data_cleaning_1.shape

(9357, 15)

In [6]:
data_cleaning_2 = data_cleaning_1.loc[:, data_cleaning_1.columns != 'NMHC(GT)']

In [7]:
data_cleaning_2['CO(GT)'] = data_cleaning_2['CO(GT)'].str.replace(',', '.').astype(float)
data_cleaning_2['C6H6(GT)'] = data_cleaning_2['C6H6(GT)'].str.replace(',', '.').astype(float)
data_cleaning_2['T'] = data_cleaning_2['T'].str.replace(',', '.').astype(float)
data_cleaning_2['RH'] = data_cleaning_2['RH'].str.replace(',', '.').astype(float)
data_cleaning_2['AH'] = data_cleaning_2['AH'].str.replace(',', '.').astype(float)

/home/ernesto/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ernesto/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ernesto/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [8]:
data_cleaning_2 = data_cleaning_2.replace(-200.000000, np.nan)

In [9]:
data_cleaning_2.dropna(thresh=10,axis=0,inplace=True)

In [10]:
data_cleaning_3 = data_cleaning_2.reset_index(drop=True)

In [11]:
no1 = data_cleaning_3[["NOx(GT)"]]

In [12]:
def outlayers_media(df,column):
    rollingdf = df.rolling(5,center=True,min_periods=1).mean()
    rollingdf = rollingdf.rename(columns={column: column+"_rolling"})
    auxdf = pd.concat([df, rollingdf],axis=1)
    auxdf['aux']=df[column].mask(pd.isnull, rollingdf[column+"_rolling"])
    final = auxdf[['aux']]
    final = final.rename(columns={'aux': column})
    return final

In [13]:
while no1[no1['NOx(GT)'].isnull()].shape[0] > 0:
    no1 = outlayers_media(no1,'NOx(GT)')

In [14]:
no2 = data_cleaning_3[["NO2(GT)"]]
while no2[no2['NO2(GT)'].isnull()].shape[0] > 0:
    no2 = outlayers_media(no2,'NO2(GT)')

In [15]:
no3 = data_cleaning_3[["CO(GT)"]]
while no3[no3['CO(GT)'].isnull()].shape[0] > 0:
    no3 = outlayers_media(no3,'CO(GT)')

In [16]:
data_cleaning_4 = data_cleaning_3.drop(['NOx(GT)', 'NO2(GT)','CO(GT)'], axis=1)

In [17]:
data_cleaning_5 = pd.concat([data_cleaning_4, no1],axis=1)
data_cleaning_5 = pd.concat([data_cleaning_5, no2],axis=1)
data_cleaning_6 = pd.concat([data_cleaning_5, no3],axis=1)

In [18]:
data_cleaning_6 = data_cleaning_6.drop(['Date', 'Time'], axis=1)

In [19]:
data = data_cleaning_6.copy()

In [20]:
X = data.loc[:, data.columns != 'NOx(GT)']
y = data[['NOx(GT)']]

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, y,random_state = 3,test_size=0.2)

In [22]:
model = LinearRegression()
model.fit(x_train, y_train)

predit_train = model.predict(x_train)
predit_test = model.predict(x_test)

print('R2 en entrenamiento es: {}'.format(model.score(x_train, y_train)))
print('R2 en test es: {}'.format(r2_score(predit_test, y_test)))

R2 en entrenamiento es: 0.8593843314298374
R2 en test es: 0.8246409221849591


In [23]:
model = LinearRegression()
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
path = "models/"
filename = 'regressor_air_quality.sav'

try:
    if not os.path.exists('models'):
        os.makedirs('models')
    pickle.dump(model, open(path+filename, 'wb'))
    print("Modelo serializado en {}".format(path+filename))
except:
    print("Error al serializar")

Modelo serializado en models/regressor_air_quality.sav
